In [1]:
!pip3 list
import pandas as pd
from datasets import Dataset
from transformers import MBartForConditionalGeneration, MBart50Tokenizer, Trainer, TrainingArguments
import torch

Package            Version
------------------ -----------
aiohappyeyeballs   2.4.3
aiohttp            3.10.10
aiosignal          1.3.1
appnope            0.1.4
asttokens          2.4.1
attrs              24.2.0
certifi            2024.8.30
charset-normalizer 3.4.0
comm               0.2.2
datasets           3.1.0
debugpy            1.8.7
decorator          5.1.1
dill               0.3.8
executing          2.1.0
filelock           3.16.1
frozenlist         1.5.0
fsspec             2024.9.0
huggingface-hub    0.26.2
idna               3.10
ipykernel          6.29.5
ipython            8.29.0
jedi               0.19.1
Jinja2             3.1.4
jupyter_client     8.6.3
jupyter_core       5.7.2
MarkupSafe         3.0.2
matplotlib-inline  0.1.7
mpmath             1.3.0
multidict          6.1.0
multiprocess       0.70.16
nest-asyncio       1.6.0
networkx           3.4.2
numpy              2.1.2
packaging          24.1
pandas             2.2.3
parso              0.8.4
pexpect            4.9.0
pi

/Users/emilysun/Downloads/btt/vscodecode/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

DATA_DIR = ''

df = pd.read_csv(f"Luganda.csv")

# Check the data
print(df.head())


                                             English  \
0  All refugees were requested to register with t...   
1     They called for a refugees' meeting yesterday.   
2  Refugees had misunderstandings between   thems...   
3  We were urged to welcome refugees into our com...   
4  More development is achieved when we work toge...   

                                             Luganda  
0  Abanoonyiboobubudamu bonna baasabiddwa beewand...  
1  Baayise olukungaana lw'abanoonyiboobubudamu eg...  
2  Abanoonyiboobubudamu b'abadde n'obutakkaanya w...  
3  Twakubirizibwa okwaniriza abanoonyiboobubudamu...  
4  Bwe tukolera awamu enkulaakulana enyingi efuni...  


In [3]:
dataset = Dataset.from_pandas(df)

In [4]:
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [5]:
model_name = "facebook/mbart-large-50"
tokenizer = MBart50Tokenizer.from_pretrained(model_name, src_lang="en_XX", tgt_lang="lg_UG")
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [6]:
def tokenize_function(examples):
    source = tokenizer(examples["English"], max_length=64, truncation=True, padding="max_length")
    target = tokenizer(examples["Luganda"], max_length=64, truncation=True, padding="max_length")

    labels = target["input_ids"]
    labels = [[label if label != tokenizer.pad_token_id else -100 for label in label_seq] for label_seq in labels]

    source["labels"] = labels
    return source

In [7]:
# Tokenize the train and test datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/12016 [00:00<?, ? examples/s]


KeyError: 'english'

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",                  # Output directory
    evaluation_strategy="epoch",             # Evaluate every epoch
    per_device_train_batch_size=4,           # Reduced batch size
    per_device_eval_batch_size=4,            # Reduced batch size for evaluation
    learning_rate=5e-5,                      # Learning rate
    num_train_epochs=2,                      # Reduced epochs
    weight_decay=0.01,                       # Strength of weight decay
    save_total_limit=1,                      # Save only the last checkpoint
    logging_dir="./logs",                    # Directory for logs
    logging_steps=1000,                      # Log less frequently to save memory
    do_train=True,
    do_eval=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  # Training dataset
    eval_dataset=tokenized_test_dataset,    # Evaluation dataset (separate test dataset)
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./fine_tuned_mBART")
tokenizer.save_pretrained("./fine_tuned_mBART")

In [ ]:
def translate(sentence):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt", max_length=128, truncation=True)
    # Generate translation using the model
    translated_tokens = model.generate(**inputs)
    # Decode the generated tokens
    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

In [ ]:
english_sentence = "All refugees were requested to register with the chairman."
luganda_translation = translate(english_sentence)
print(f"English: {english_sentence}")
print(f"Luganda: {luganda_translation}")